In [3]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import threading
import queue
import re

def response_func(query,num):
    query = query.replace(' ','+')
    url = f'https://www.amazon.com/s?k={query}&page={num}'
    headers = {'accept-encoding':'gzip, deflate, br, zstd','accept-language':'en-US,en;q=0.9','user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'}
    response = requests.get(url,headers=headers)
    if response.status_code==200:
        return response.text
    else:
        print('Check your connection')

def open_url(url):
    headers = {'accept-encoding':'gzip, deflate, br, zstd','accept-language':'en-US,en;q=0.9','user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'}
    response_data = requests.get(url,headers=headers)
    soup_2 = bs(response_data.text,'html.parser')
    return soup_2

def extract_page(query,num,result_queue):
    data_list = []
    response = response_func(query,num)
    soup = bs(response,'html.parser')
    big_box = soup.find_all('div',{'role':'listitem'})
    for box in big_box:
        link = box.find('a').get('href')
        url = f'https://www.amazon.com{link}'
        page = open_url(url)
        try:
            element = page.find(attrs={'data-asin': True})
            asin = element['data-asin']
            name_text = page.find('h1',{'id':'title'}).text
            name = re.sub(r'\s+', ' ', name_text).strip()
            regular_price = page.find('span',{'class':"a-price aok-align-center reinventPricePriceToPayMargin priceToPay"}).text
            discounted_price_ele = page.find('span',{"class":"a-size-small aok-offscreen"})
            if discounted_price_ele:
                discounted_price = discounted_price_ele.text.split(':')[1]
            else:
                discounted_price = None
            feature_ele = page.find('div',{'id':"feature-bullets"})
            if feature_ele:
                features = feature_ele.find('ul').text
            else:
                features= page.find('div',{'id':"productFactsDesktopExpander"}).find('ul').text
            image = page.find('div',{'id':"imgTagWrapperId"}).find('img').get('src')
            rating = page.find('div',{'id':"averageCustomerReviews"}).find('span',{'class':'a-size-base a-color-base'}).text
            reviews = page.find('span',{'id':"acrCustomerReviewText"}).text.replace('ratings',"")
            supplier_text = page.find('div',{'id':"bylineInfo_feature_div"}).text.replace('Visit the ',"").replace('Brand: ',"")
            supplier = re.sub(r'\s+', ' ', supplier_text).strip()
            supplier_url = page.find('div',{'id':'bylineInfo_feature_div'}).find('a').get('href')
            dict_data = {'ASIN': asin,
                        'Name':name,
                        'Price':regular_price,
                        'List Price':discounted_price,
                        'Features':features,
                        'Image':image,
                        'Rating':rating,
                        'Reviews':reviews,
                        'Supplier':supplier,
                        'Supplier_url':f'https://www.amazon.com{supplier_url}'}
            data_list.append(dict_data)
        except AttributeError:
            print('data not found')
            continue
    result_queue.put(data_list)

def extract_data(query):
    times = 8
    threads = []
    result_queue = queue.Queue()
    for i in range(times):
        thread = threading.Thread(target=extract_page,args=(query,i,result_queue))
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join()
    
    total_data = []
    while not result_queue.empty():
        total_data.extend(result_queue.get())  # Combine the data lists from all threads

    return total_data

def convert_csv(data,filename):
    df = pd.DataFrame(data)
    df.to_csv(filename)

In [5]:
convert_csv(data,'Amazon_data.csv')

In [4]:
data = extract_data('Men Watch')

data not found
data not found
data not found
data not found
data not found
data not found
data not found
data not found


Exception in thread Thread-21 (extract_page):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/Users/hydermac/Library/Python/3.12/lib/python/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/6k/9lcwl8q960scjdl32lqz8h_80000gn/T/ipykernel_34884/2597064945.py", line 41, in extract_page
IndexError: list index out of range


data not found


Exception in thread Thread-16 (extract_page):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/Users/hydermac/Library/Python/3.12/lib/python/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/6k/9lcwl8q960scjdl32lqz8h_80000gn/T/ipykernel_34884/2597064945.py", line 41, in extract_page
IndexError: list index out of range
Exception in thread Thread-15 (extract_page):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/Users/hydermac/Library/Python/3.12/lib/python/site-packages/ipykernel/ipkernel.py", line 766, in run_closu

data not found
data not found
data not found
data not found
data not found
data not found
data not found
data not found
data not found
data not found
data not found
data not found
data not found
data not found
data not found
data not found
data not found
data not found
data not found
data not found
data not found
data not found
data not found
data not found
data not found


In [6]:
data.shape

AttributeError: 'list' object has no attribute 'shape'

In [7]:
data = pd.read_csv('Amazon_data.csv')
data

,Unnamed: 0,ASIN,Name,Price,List Price,Features,Image,Rating,Reviews,Supplier,Supplier_url
0,0,B0DHZ6CNDL,Military Smart Watch for Men(Answer/Dial Calls...,$72.99,NaN,➤【Military Grade】SUNKTA smart watch breaks a...,https://m.media-amazon.com/images/I/71pc1LaYUF...,5.0,6,SUNKTA Store,https://www.amazon.com/stores/SUNKTA/page/5C6D...
1,1,B07WLJWMHM,BENYAR Mens Watches Chronograph Date Waterproo...,$39.98,NaN,\n\n \nClasp type \n \nBuckle \n \n\n \nCo...,https://m.media-amazon.com/images/I/81bZU86djq...,4.5,"2,024",CEYADG Store,https://www.amazon.com/stores/CEYADG/page/4FC7...
2,2,B0DK3KQ55X,"Pagani Design Watches for Men, 38mm Dial, Stai...",$69.99,NaN,\n\n \nClasp type \n \nPush Button Foldover ...,https://m.media-amazon.com/images/I/71PWwg5nXy...,4.7,11,Orizontime Store,https://www.amazon.com/stores/Orizontime/page/...
3,3,B09P1GHH3M,FORSINING Vintage Watch for Men Engraved Autom...,$41.99,NaN,\n\n \nItem width \n \n11 centimeters \n \n...,https://m.media-amazon.com/images/I/81jjRDkzcd...,4.2,98,FORSINING Store,https://www.amazon.com/stores/FORSINING/page/A...
4,4,B07JZFQQYL,"Invicta Mens Pro Diver Quartz Watch, Black, 28001",$64.90,NaN,"Black dial, Black silicone band 50mm stainles...",https://m.media-amazon.com/images/I/7135J0dg1g...,4.7,50,Invicta Store,https://www.amazon.com/stores/InvictaStores/pa...
...,...,...,...,...,...,...,...,...,...,...,...
262,262,B0733PT6VC,Bulova Men's Marine Star Series C Stainless St...,$256.00,$575.00,\n\n \nMaterial \n \nStainless-steel \n \n\...,https://m.media-amazon.com/images/I/81GbXYQRp8...,4.6,499,Bulova Store,https://www.amazon.com/stores/BulovaCorporatio...
263,263,B09VGFS21M,"FILIEKEU Men Smart Watch for Android iOS, Blue...",$49.99,NaN,\n\n \nItem width \n \n4.2 centimeters \n \...,https://m.media-amazon.com/images/I/615S7jF9tT...,4.0,"1,657",FILIEKEU,https://www.amazon.com/FILIEKEU/b/ref=bl_sl_s_...
264,264,B0BTBX691N,"Smart Watch for Men(Answer/Make Call),Alexa Bu...",$77.99,NaN,【Make/Answer Call】After connecting to your p...,https://m.media-amazon.com/images/I/61IiAwla6H...,4.1,"7,145",Woneligo Store,https://www.amazon.com/stores/WoneligoSmartwat...
265,265,B07G5Q8LKH,Bulova Men's Crystal Octava Chronograph Quartz...,$301.96,$595.00,"From the Bulova Men's Crystal Collection, mak...",https://m.media-amazon.com/images/I/71Yb3ng2PB...,4.7,"1,852",Bulova Store,https://www.amazon.com/stores/BulovaCorporatio...
